# 可命名元组

In [26]:
from collections import  namedtuple

In [27]:
date_array = ['20170118', '20170119','20170120','20170121']
price_array = ['30.1', '30.2','31.2','32.3']
stock_nametuple = namedtuple('stock',('date', 'price'))
stock_namedtuple_list = [stock_nametuple(date, price) for
                        date, price in zip(date_array, price_array)]
stock_namedtuple_list

[stock(date='20170118', price='30.1'),
 stock(date='20170119', price='30.2'),
 stock(date='20170120', price='31.2'),
 stock(date='20170121', price='32.3')]

# 有序字典 OrderedDict

In [28]:
from collections import  OrderedDict

In [29]:
stock_dict = OrderedDict(
    (date, price) for date, price in zip(date_array, price_array))
stock_dict

OrderedDict([('20170118', '30.1'),
             ('20170119', '30.2'),
             ('20170120', '31.2'),
             ('20170121', '32.3')])

# lambda函数

In [30]:
find_second_max_lambad = lambda dict_array: sorted(
    zip(dict_array.values(), dict_array.keys()))[-2]
find_second_max_lambad(stock_dict)

('31.2', '20170120')

# 高阶函数

* map(), 接收两个参数，一个函数，一个序列
* filter()， 两个参数，一个函数，一个序列
* reduce()， 函数作用在一个序列上，这个函数必须接收两个参数。

In [31]:
# 计算涨跌幅
price_float_array = [float(price_str) for price_str in 
                    stock_dict.values()]
pp_array = [(price1, price2) for price1,price2 in 
           zip(price_float_array[0:-1], price_float_array[1:])]
pp_array

[(30.1, 30.2), (30.2, 31.2), (31.2, 32.3)]

In [32]:
from functools import reduce
# 把结果继续和序列的下一个元素做累积计算
change_array = map(
    lambda pp: reduce(lambda a,b: round((b-a)/a, 3), pp), pp_array)
list(change_array)

[0.003, 0.033, 0.035]

# 面向对象

In [56]:
from collections import  namedtuple
from collections import  OrderedDict
from functools import reduce

# 交易对象
class StockTradeDays(object):
    def __init__(self, price_array, start_date, date_array=None):
        # 价格
        self.__price_array = price_array
        # 日期序列
        self.__date_array = self._init_days(start_date, date_array)
        # 涨跌幅序列
        self.__change_array = self.__init_change()
        # OrderedDict 的组装
        self.stock_dict = self._init_stock_dict()
        
    def __init_change(self):
        """
        用price_array 生成 change_array
        """
        price_float_array = [float(price_str) for price_str in self.__price_array]
        pp_array = [(p1,p2) for p1,p2 in zip(price_float_array[:-1], price_float_array[1:])]
        change_array = list(map(lambda pp: reduce(lambda a,b : round((b-a)/a, 3), pp), pp_array))
        return change_array
    
    def _init_days(self, start_date, date_array):
        """
            protect 方法
        """
        if date_array is None:
            date_array = [str(start_date + ind) for ind,_ in enumerate(self.__price_array)]
            date_array = [str(date) for date in date_array]
        else:
            date_aray = [str(date) for date in date_array]
        return date_array
        
    def _init_stock_dict(self):
        """
        使用 namedtuple, OrderedDict 将结果合并
        """
        # 命名元组
        """
        collections.namedtuple(typename, field_names, verbose=False, rename=False)
        返回一个命名元祖子类typename，其中参数的意义如下： 
        typename,：此元组的名称； 
        field_names: 元祖中元素的名称（类似于c结构体中的age等），此字段有多种表达方式，见例子； 
        rename:如果元素名称中含有python的关键字，则必须设置为rename=True,具体见下面； 
        verbose:默认就好；
        """
        stock_namedtuple = namedtuple('stock',('date', 'price', 'change'))
        # 有序字典
        stock_dict = OrderedDict((date, stock_namedtuple(date, price, change)) for date, price, change in zip(self.__date_array, self.__price_array, self.__change_array))
        return stock_dict
    
    # 筛选结果子集
    def filter_stock(self, want_up=True, want_calc_sum=False):
        """
        want_up 是否刷选上涨
        want_cals_sum : 是否计算涨跌幅
        """
        filter_fuc = (lambda day: day.change > 0) if want_up else (lambda day: day.change < 0)
        want_days = filter(filter_fuc, self.stock_dict.values())
        if not want_calc_sum:
            return want_days
        change_sum = 0.0
        for day in want_days:
            change_sum += day.change
        return change_sum
    """
    __str__,  __iter__,  __getitem__, __len__ 内建函数
    """
    def __str__(self):
        return str(self.stock_dict)
    
    def __iter__(self):
        for key in self.stock_dict:
            yield self.stock_dict[key]
        
    def __getitem__(self, ind):
        date_key = self.__date_array[ind]
        return self.stock_dict[date_key]
    
    def __len__(self):
        return len(self.stock_dict)

In [34]:
price_array = '30.14, 29.58, 26.36, 32.56, 32.82'.split(',')
date_base = 20170118
trade_days = StockTradeDays(price_array, date_base)
print(trade_days)

OrderedDict([('20170118', stock(date='20170118', price='30.14', change=-0.019)), ('20170119', stock(date='20170119', price=' 29.58', change=-0.109)), ('20170120', stock(date='20170120', price=' 26.36', change=0.235)), ('20170121', stock(date='20170121', price=' 32.56', change=0.008))])


In [35]:
len(trade_days)

4

In [36]:
from collections import Iterable
if isinstance(trade_days, Iterable):
    for day in trade_days:
        print("     " + str(day))

     stock(date='20170118', price='30.14', change=-0.019)
     stock(date='20170119', price=' 29.58', change=-0.109)
     stock(date='20170120', price=' 26.36', change=0.235)
     stock(date='20170121', price=' 32.56', change=0.008)


In [37]:
trade_days.filter_stock()

In [38]:
from abupy import ABuSymbolPd

In [39]:
# 获取2年的 TSLA 的收盘价
price_array = ABuSymbolPd.make_kl_df('TSLA', n_folds=2).close.tolist()
date_array = ABuSymbolPd.make_kl_df('TSLA', n_folds=2).date.tolist()
price_array[:5],date_array[:5]

([191.17, 193.14, 196.65, 196.12, 189.57],
 [20161122, 20161123, 20161125, 20161128, 20161129])

In [40]:
trade_days = StockTradeDays(price_array, date_base, date_array)
len(trade_days)

502

In [41]:
trade_days[-2]

stock(date=20181119, price=358.88, change=-0.033)

## 继承与多态

In [42]:
import six 
from abc import ABCMeta, abstractmethod

In [43]:
class TradeStrategyBase(six.with_metaclass(ABCMeta, object)):
    """
    交易策略
    """
    # 声明为借口
    @abstractmethod
    def buy_strategy(self, *args, **kwagrs):
        pass
    @abstractmethod
    def sell_strategy(self, *args, **kwargs):
        pass

In [48]:
# 策略类
class TradeStrategy1(TradeStrategyBase):
    """
    追涨策略： 当股票上涨大于7% 时买入
    """
    s_keep_stock_threshold = 20
    def __init__(self):
        self.keep_stcok_day = 0
        self.__buy_change_threshold = 0.07
    
    def buy_strategy(self, trade_ind, trade_day, trade_days):
        if self.keep_stcok_day == 0 and trade_day.change > self.__buy_change_threshold:
            self.keep_stcok_day +=1
        elif self.keep_stcok_day > 0:
            self.keep_stcok_day +=1
            
    def sell_strategy(self, trade_ind, trade_day, trade_days):\
        # 持股大于持股阈值，卖出股票
        if self.keep_stcok_day >= TrateStrategy1.s_keep_stock_threshold:
            self.keep_stcok_day = 0
            
    @property
    def buy_change_threshold(self):
        return self.__buy_change_threshold
    
    @buy_change_threshold.setter
    def buy_change_threshold(self, buy_change_threshold):
        if not isinstance(buy_change_threshold, float):
            """
            上涨阈值为float类型
            """
            raise TypeError('buy_change_threshold must be float')
        self.__buy_change_threshold = round(buy_change_threshold, 2)
        
# 交易回测类
class TradeLoopBack(object):
    def __init__(self, trade_days, trade_strategy):
        self.trade_days = trade_days
        self.trade_strategy = trade_strategy
        self.profit_array = []
    
    def execute_trade(self):
        for ind, day in enumerate(self.trade_days):
            if self.trade_strategy.keep_stock_day > 0:
                self.profit_array.append(day.change)
            if hasattr(self.trade_strategy, 'buy_strategy'):
                # 买入策略执行
                self.trade_strategy.buy_strategy(ind, day, self.trade_days)
            if hasattr(self.trade_strategy, 'sell_strategy'):
                self.trade_strategy.sell_strategy(ind, day, self.trade_days)
                

              

In [62]:
from functools import  reduce

In [63]:
 # 两年的TSLA收盘数据 to list
price_array = ABuSymbolPd.make_kl_df('TSLA', n_folds=2).close.tolist()
# 两年的TSLA收盘日期 to list，这里的写法不考虑效率，只做演示使用
date_array = ABuSymbolPd.make_kl_df('TSLA', n_folds=2).date.tolist()

trade_days = StockTradeDays(price_array, 0, date_array)
trade_loop_back = TradeLoopBack(trade_days, TradeStrategy1())
print(trade_loop_back.profit_array)
#print('交易策略1总盈亏为:{}%'.format(reduce(lambda a,b: a+b, trade_loop_back.profit_array))*100)

[]


# 性能与效率
## itertool 的使用

In [64]:
import itertools

In [67]:
items = [1,2,3]
# 顺序组合元素
for item in itertools.permutations(items):
    print("    " + str(item))

    (1, 2, 3)
    (1, 3, 2)
    (2, 1, 3)
    (2, 3, 1)
    (3, 1, 2)
    (3, 2, 1)


In [68]:
# 不考虑顺序，不放回数据
for item in itertools.combinations(items, 2):
    print("  " + str(item))

  (1, 2)
  (1, 3)
  (2, 3)


In [69]:
# 不考虑顺序，有放回
for item in itertools.combinations_with_replacement(items, 2):
    print("  " + str(item))

  (1, 1)
  (1, 2)
  (1, 3)
  (2, 2)
  (2, 3)
  (3, 3)


In [70]:
# 笛卡尔积
ab = ['a', 'b']
cd = ['c', 'd']
for item in itertools.product(ab, cd):
    print('  ' + str(item))

  ('a', 'c')
  ('a', 'd')
  ('b', 'c')
  ('b', 'd')


## 多进程与多线程

### 1. 使用多进程

In [72]:
from concurrent.futures import ProcessPoolExecutor
result = []
def when_donw(r):
    result.append(r.result())
    with ProcessPoolExecutor as pool:
        for keep_stock_threshold, buy_change_threshold in itertools.product(keep_stock_list, buy_change_list):
            future_result = pool.submit(calc, keep_stock_threshold, buy_change_threshold)
            future_result.add_done_callback(when_donw)
